In [ ]:
# | default_exp classes.DomoDataflow

In [ ]:
# | exporti
from __future__ import annotations

In [ ]:
# | exporti

from dataclasses import dataclass, field
from typing import List

import domolibrary.utils.DictDot as util_dd
import domolibrary.utils.chunk_execution as ce

import domolibrary.client.DomoAuth as dmda
import domolibrary.routes.dataflow as dataflow_routes

import httpx
from nbdev.showdoc import patch_to

In [ ]:
# | export
from domolibrary.classes.DomoDataflow_Action import DomoDataflow_Action
from domolibrary.classes.DomoDataflow_History import (
    DomoDataflow_History,
    DomoDataflow_ActionResult,
)

In [ ]:
# | hide
from nbdev.showdoc import show_doc

In [ ]:
# | export
@dataclass
class DomoDataflow:
    id: str
    name: str = None
    auth: dmda.DomoAuth = field(default=None, repr=False)
    owner: str = None
    description: str = None
    tags: list[str] = None
    actions: list[DomoDataflow_Action] = None

    version_id: int = None
    version_number: int = None
    versions : List[dict] = None # list of DomoDataflow Versions

    history: DomoDataflow_History = None  # class for managing the history of a dataflow

    def __post_init__(self):
        self.history = DomoDataflow_History(
            dataflow=self, dataflow_id=self.id, auth=self.auth
        )


    @classmethod
    def _from_json(cls, obj, auth, version_id=None, version_number=None):
        dd = util_dd.DictDot(obj)

        domo_dataflow = cls(
            auth=auth,
            id=dd.id,
            name=dd.name,
            description=dd.description,
            owner=dd.owner,
            tags=dd.tags,
            version_id=version_id,
            version_number=version_number,
        )

        if dd.actions:
            domo_dataflow.actions = [
                DomoDataflow_Action._from_json(action) for action in dd.actions
            ]

            [ domo_action.get_parents(domo_dataflow.actions) for domo_action in domo_dataflow.actions]

        return domo_dataflow

In [ ]:
# | exporti
@patch_to(DomoDataflow, cls_method=True)
async def get_from_id(
    cls: DomoDataflow,
    dataflow_id: int,
    auth: dmda.DomoAuth = None,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    res = await dataflow_routes.get_dataflow_by_id(
        auth=auth,
        dataflow_id=dataflow_id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=cls.__name__,
        session=session,
    )

    if return_raw:
        return res

    if not res.is_success:
        return None

    return cls._from_json(res.response, auth=auth)

#### sample implementation of get_from_id


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoDataflow.get_from_id(dataflow_id=70, auth=token_auth)

DomoDataflow(id=70, name='DomoStats - people - reports', owner=None, description=None, tags=None, actions=[DomoDataflow_Action(id='241025d7-3cca-4369-b7c0-b3264277c0e1', type='LoadFromVault', depends_on=None, name='domostats_people', datasource_id='241025d7-3cca-4369-b7c0-b3264277c0e1', sql=None, parent_actions=None), DomoDataflow_Action(id='c176aa53-122a-4a2e-8b6f-41ca4a72d67c', type='GenerateTableAction', depends_on=['241025d7-3cca-4369-b7c0-b3264277c0e1'], name='t_1', datasource_id=None, sql='SELECT\n`Display Name`,\n`User ID`\n, @row := @row+1 as id\n, @isReset := case when @iter = @rowCounter then 1 else 0 end isReset\n, @group := @isReset + @group   as rowGroup\n, @iter := case when @isReset = 1 then 1 else @iter + 1 end iter\n, @rowCounter := case when @isReset = 1 then FLOOR( RAND() * (7-3) + 3) else @rowCounter end numChildren\n\n\nfrom `domostats_people` p\n\n, (select @row:= 0,@rowCounter:= 1, @iter := 1, @isReset := 0 , @group := 0) r', parent_actions=[DomoDataflow_Action(i

#### sample retrieval that also interacts with dataflow history from `DomoDataflow_History`

note will return actions history


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataflow = await DomoDataflow.get_from_id(dataflow_id=70, auth=token_auth)
await domo_dataflow.history.get_execution_history(maximum=5)

[DomoDataflow_History_Execution(id=436301, dataflow_id=70, dataflow_execution_id='39850865-7669-4897-bc21-96a3bb66589f', dataflow_version=262, begin_time=datetime.datetime(2023, 12, 8, 21, 6, 53), end_time=datetime.datetime(2023, 12, 8, 21, 7, 25), last_updated=datetime.datetime(2023, 12, 8, 21, 7, 25), is_failed=False, state='SUCCESS', activation_type='MANUAL', data_processor='MYSQL', telemetry=None, execution_stats={'total_bytes_written': 98190, 'total_rows_read': 911, 'total_bytes_read': 291865, 'mean_download_rate_kbps': 60.604808433446735, 'total_rows_written': 1820}, action_results=[DomoDataflow_ActionResult(id='8edc3d78-3586-47a8-9775-c7b146a50d19', type='PublishToVault', depends_on=None, name=None, is_success=True, rows_processed=1820, begin_time=datetime.datetime(2023, 12, 8, 21, 7, 23, 501000), end_time=datetime.datetime(2023, 12, 8, 21, 7, 24, 635000), duration_in_sec=1.134), DomoDataflow_ActionResult(id='c176aa53-122a-4a2e-8b6f-41ca4a72d67c', type='GenerateTableAction', dep

In [ ]:
# | exporti
@patch_to(DomoDataflow)
async def execute(
    self: DomoDataflow,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    return await dataflow_routes.execute_dataflow(
        auth=auth or self.auth,
        dataflow_id=self.id,
        parent_class=self.__class__.__name__,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

#### sample execute_dataflow


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataflow = DomoDataflow(id=70, auth=token_auth)

await domo_dataflow.execute()

ResponseGetData(status=200, response={'id': 436301, 'onboardFlowId': 70, 'previewRows': 0, 'dapDataFlowExecutionId': '39850865-7669-4897-bc21-96a3bb66589f', 'beginTime': 1702069612558, 'lastUpdated': 1702069613275, 'state': 'CREATED', 'activationType': 'MANUAL', 'executionEngine': {'platform': 'K8S', 'engine': 'MYSQL_SQL_ENGINE'}, 'dataProcessor': 'MYSQL', 'dataFlowVersion': 262}, is_success=True, parent_class=None)

# Dataflow Versions


In [ ]:
# | exporti


@patch_to(DomoDataflow, cls_method=True)
async def get_by_version_id(
    cls: DomoDataflow,
    auth: dmda.DomoAuth,
    dataflow_id: int,
    version_id: int,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    res = await dataflow_routes.get_dataflow_by_id_and_version(
        auth=auth,
        dataflow_id=dataflow_id,
        version_id=version_id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=cls.__name__,
        session = session
    )

    if return_raw:
        return res

    domo_dataflow = cls._from_json(
        res.response["dataFlow"],
        version_id=res.response["id"],
        version_number=res.response["versionNumber"],
        auth=auth,
    )

    return domo_dataflow

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoDataflow.get_by_version_id(
    dataflow_id=70, version_id=262, auth=token_auth, return_raw=False
)

DomoDataflow(id=70, name='DomoStats - people - reports', owner=None, description=None, tags=None, actions=[DomoDataflow_Action(id='241025d7-3cca-4369-b7c0-b3264277c0e1', type='LoadFromVault', depends_on=None, name='domostats_people', datasource_id='241025d7-3cca-4369-b7c0-b3264277c0e1', sql=None, parent_actions=None), DomoDataflow_Action(id='c176aa53-122a-4a2e-8b6f-41ca4a72d67c', type='GenerateTableAction', depends_on=['241025d7-3cca-4369-b7c0-b3264277c0e1'], name='t_1', datasource_id=None, sql='SELECT\n`Display Name`,\n`User ID`\n, @row := @row+1 as id\n, @isReset := case when @iter = @rowCounter then 1 else 0 end isReset\n, @group := @isReset + @group   as rowGroup\n, @iter := case when @isReset = 1 then 1 else @iter + 1 end iter\n, @rowCounter := case when @isReset = 1 then FLOOR( RAND() * (7-3) + 3) else @rowCounter end numChildren\n\n\nfrom `domostats_people` p\n\n, (select @row:= 0,@rowCounter:= 1, @iter := 1, @isReset := 0 , @group := 0) r', parent_actions=[DomoDataflow_Action(i

In [ ]:
# | exporti
@patch_to(DomoDataflow)
async def get_versions(
    self: DomoDataflow,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    res = await dataflow_routes.get_dataflow_versions(
        auth=self.auth, dataflow_id=self.id,
        debug_api = debug_api, session = session,
        debug_num_stacks_to_drop = debug_num_stacks_to_drop,
        parent_class = self.__class__.__name__
    )

    if return_raw:
        return res

    version_ids = [df_obj["id"] for df_obj in res.response]

    self.versions= await ce.gather_with_concurrency(
        *[
            DomoDataflow.get_by_version_id(
                dataflow_id=self.id, version_id=version_id, auth=self.auth,
                session = session,
                debug_api = debug_api,
                debug_num_stacks_to_drop = debug_num_stacks_to_drop

            )
            for version_id in version_ids
        ],
        n=10
    )

    return self.versions

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataflow = DomoDataflow(id=70, auth=token_auth)

await domo_dataflow.get_versions(return_raw=False)

[DomoDataflow(id=70, name='DomoStats - people - reports', owner=None, description=None, tags=None, actions=[DomoDataflow_Action(id='241025d7-3cca-4369-b7c0-b3264277c0e1', type='LoadFromVault', depends_on=None, name='domostats_people', datasource_id='241025d7-3cca-4369-b7c0-b3264277c0e1', sql=None, parent_actions=None), DomoDataflow_Action(id='c176aa53-122a-4a2e-8b6f-41ca4a72d67c', type='GenerateTableAction', depends_on=['241025d7-3cca-4369-b7c0-b3264277c0e1'], name='t_1', datasource_id=None, sql='SELECT\n`Display Name`,\n`User ID`\n, @row := @row+1 as id\n, @isReset := case when @iter = @rowCounter then 1 else 0 end isReset\n, @group := @isReset + @group   as rowGroup\n, @iter := case when @isReset = 1 then 1 else @iter + 1 end iter\n, @rowCounter := case when @isReset = 1 then FLOOR( RAND() * (7-3) + 3) else @rowCounter end numChildren\n\n\nfrom `domostats_people` p\n\n, (select @row:= 0,@rowCounter:= 1, @iter := 1, @isReset := 0 , @group := 0) r', parent_actions=[DomoDataflow_Action(

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()